
## CNN With Custom Images

The dataset contains 2 folders - Infected - Uninfected

And a total of 27,558 images.


This Dataset is taken from the official NIH Website: https://ceb.nlm.nih.gov/repositories/malaria-datasets/ 

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pwd

In [ ]:
my_data_dir = "C:\\Users\APO\Desktop\cell_images\cell_images"

In [ ]:
# CONFIRM THAT THIS REPORTS BACK 'test', and 'train'
os.listdir(my_data_dir) 

In [ ]:
test_path = my_data_dir+'\\test\\'  # test ve train dosyalarıma ulaştım 
train_path = my_data_dir+'\\train\\'

In [ ]:
os.listdir(test_path) # ilgili dosyaya çift tıklama gibi düşünebiliriz 

In [ ]:
os.listdir(train_path)

In [ ]:
os.listdir(train_path+'\\parasitized') # ilgili klasördeki tüm dosyalar

In [ ]:
os.listdir(train_path+'\\parasitized')[9] # 9.yu aldı

In [ ]:
para_cell = train_path+'\\parasitized'+'\\C100P61ThinF_IMG_20150918_144104_cell_171.png'

In [ ]:
imread(para_cell)

# parasitized klasöründeki 9. indeksteki resmin matrislerini görüyorum.

In [ ]:
para_img= imread(para_cell)

# bu image in matrislerini rahat kullanabilmek için bir değişkene atadım.

In [ ]:
plt.imshow(para_img)

# bu image'i imshow fonksiyonu ile görebiliyorum. Bu enfekte olmuş bir hücre fotoğrafı

In [ ]:
para_img.shape

# 142 ye 139 pikselden oluşan ve renkli (3) bir image

In [ ]:
os.listdir(train_path+'\\uninfected')

In [ ]:
# bir de enfekte olmayan bir hücre görünümüne bakalım.

uninfected_cell_path = train_path+'\\uninfected\\'+os.listdir(train_path+'\\uninfected')[7]

In [ ]:
imread(uninfected_cell_path)

In [ ]:
unifected_cell = imread(uninfected_cell_path)
plt.imshow(unifected_cell) # enfekte olmamış bir hücreye baktım

**Let's check how many images there are.**

In [ ]:
len(os.listdir(train_path+'\\parasitized'))

# enfekte olmuş (parasitized) 12480 image mevcut.

In [ ]:
len(os.listdir(train_path+'\\uninfected'))

# enfekte olmmmış (uninfected) 12480 image mevcut.

**Let's find out the average dimensions of these images.**

In [ ]:
unifected_cell.shape

# 136 ye 148 pikselden oluşan ve renkli (3) bir image

In [ ]:
para_img.shape

# pikselleri dolayısıyla boyutları farklı. 
# Bu şekilde modele sokamam. hepsinin aynı büyüklükte olması lazım. 

In [ ]:
 # klasördeki resimlerin x ve y eksen değerlerini buluyorum
x = [] 
y = []
for image in os.listdir(test_path+'\\uninfected'):
    
    img = imread(test_path+'\\uninfected'+'\\'+image)
    d1,d2,colors = img.shape    # tuple unpacking yaparak d1,d2 ve colors'a atama yapıyorum
    x.append(d1)
    y.append(d2)
    
# uninfected klasöründeki tüm image leri inceleycek ve bana tüm x ve y değerlerini liste olarak getirecek

In [ ]:
y

In [ ]:
sns.scatterplot(x,y)

# scatterplot ile boyuların dağılımına bakıyorum. 
# x ve y nin ortalama piksel boyutu 130 gibi.

In [ ]:
np.mean(x)

# x lerin piksel ortalaması 130.92

In [ ]:
np.mean(y)

# y lerin piksel ortalaması 130.75

In [ ]:
image_shape = (130,130,3)

#  tüm fotoğrafların boyutunu ortalama değere yakın bir değere eşitledim.

#  130a 130 dan küçük olan imagelerin kenarlarını 0 lar ile dolduracak
# bundan büyük olan imageleri 130a 130 olacak şekilde kırpacak.
# bu durumda enfekte olmuş alan kenar bölgelere yakın ise kırpıldığında bunun atılma riski var .
# imagelerde kenarlarda enfekte alanlar olup olmadığına bakılıp boyutu bu alanlar içeride kalacak şekilde ayarlanabilir.

## Preparing the Data for the model

In [ ]:
# daha önce bu kodları kullanarak uninfected_cell değişkenine atama yapmıştık.

# uninfected_cell_path = train_path+'\\uninfected\\'+os.listdir(train_path+'\\uninfected')[7]
# unifected_cell = imread(uninfected_cell_path)

In [ ]:
unifected_cell.max()

In [ ]:
para_img.max()

# maksimum değerlere bakarsak datamı scale etmeme gerek yok.

In [ ]:
# dl de temel mantık cok fazla oğrenme işlemi gercekleştirmek oldugu için , 
# elimizde yeteri kadar image datası yok ise model bundan bi şey ogreneemez
# bu noktada aynı imajın farklı varysyonlarını uretıp modelimin öğrenmesini pekiştirebiliriz
# fake data girerek bunu saglayabiliriz

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#help(ImageDataGenerator)

In [ ]:
# datamızı aşağıda belirtilen parametreler ölçüsünde değiştirerek (genelde %10'ar) yeni imageler oluşturup çoğaltıyoruz.

image_gen = ImageDataGenerator(rotation_range=15, 
                               width_shift_range=0.10, # bu oranda genişliği büyütüp/küçültüyor. Burada %10 büyütüyor
                               height_shift_range=0.10, # bu oranda yüksekliği büyütüp/küçültüyor
                               #rescale=1/255,  # scale yapma ihtiyacı olsaydı...
                               shear_range=0.1,  # resmin bir kısmını almak.(random olarak numune alıyor)
                               zoom_range=0.1,   # bu oranda zoom ediyor
                               horizontal_flip=True, 
                               fill_mode='nearest')  
# fill_mode='nearest': shear edip bir kısmını alınca imagein geri kalanını yakındaki (nearest) noktalar ile doldur. 

In [ ]:
plt.imshow(para_img)

In [ ]:
plt.imshow(image_gen.random_transform(para_img))

# yukardaki imagei image_gen de belirttiğim koda göre kırpıp ilave etti.

In [ ]:
image_gen.flow_from_directory(train_path) # bizim belirlediğimiz flow directoryye akış yapıyor

In [ ]:
image_gen.flow_from_directory(test_path)

# Creating the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

# en istikrarlısı ve hızlı öğrenme sağlayan relu olduğu için relu aktivasyonunu seçtim.

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu')) # model daha hızlı oğrenıyor

model.add(Dropout(0.5)) # nöronların yarısını kapatıyor

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
''' dense_2 (Dense) satırına baktığımızda çok büyük sayıda 1605760 parametrem var. '''
''' Bu sistemimi epey yoracak. '''

## Early Stopping

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

## Training the Model

In [ ]:
#help(image_gen.flow_from_directory)

In [ ]:
image_shape

In [ ]:
# yukarda yeni değiştirilmiş datalarla imageleri çoğaltmıştık ancak bunu kullanmamıştık. 
  # burada ise batch_size ile her bir fotoğraftan 15 tane daha (yukarda belirtilen özellikleri kullanarak)  
    # üretip bunları RAM'de tutuyor. 
     # ne zamanki fit etmeye başlayacak, o zaman yeni imageleri tek tek üretip eğitime sokacak, üretip eğitime sokacak. 

batch_size = 16
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2], # target size'ı 130a 130 al demiş oldum.
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

# RAM'ine güveniyorsan (16 GB ve üzeri ise) bach_size ı büyük seçerbilirsin. yoksa RAM patlar.
# RAM düşükse bu değeri 8 olarak belirleyebilirsin. biraz uzun sürer ama sistemi yormaz.

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

# shuffle = False -->  test image ları karıştırma. 
# onlar bir sıra dahilinde olduğu için ve test'te bunların sırası önemli olduğu için 
   # bunların sırasının bozulmasını istemiyorum.

In [ ]:
train_image_gen.class_indices

# yukarda class_mode='binary' dediğim için:
  # binary mode dan öğrenerek enfekte olanlara 0 olmayalara 1 class ı atamış

In [ ]:
results = model.fit_generator(train_image_gen,epochs=20,
                              validation_data=test_image_gen, callbacks=[early_stop])

# fit_generatör'ü kullanarak yukarda ürettiğim test_image_gen ve train_image_gen directory lerden aldıklarını içindeki parametrelere göre modeli fit et. 

fit_generatör'ü kullanarak yukarda ürettiğim test_image_gen ve train_image_gen directory lerden aldıklarını içindeki parametrelere göre modeli fit et. 

# Evaluating the Model

In [ ]:
summary = pd.DataFrame(model.history.history)
summary.head()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.loss, label="loss")
plt.plot(summary.val_loss, label="val_loss")
plt.legend(loc="upper right")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.accuracy, label="accuracy")
plt.plot(summary.val_accuracy, label="val_accuracy")
plt.legend(loc="upper left")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

In [ ]:
model.metrics_names

In [ ]:
model.evaluate_generator(test_image_gen)

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
preds = model.predict_generator(test_image_gen)

In [ ]:
preds

In [ ]:
test_image_gen.classes

In [ ]:
predictions = preds > 0.5

In [ ]:
predictions

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
confusion_matrix(test_image_gen.classes,predictions)

In [ ]:
#model.save('malaria_model.h5')

# Predicting on an Image

In [ ]:
para_cell

In [ ]:
my_image = image.load_img(para_cell,target_size=image_shape)

In [ ]:
my_image

In [ ]:
type(my_image)

In [ ]:
my_image = image.img_to_array(my_image)

In [ ]:
type(my_image)

In [ ]:
my_image.shape

In [ ]:
my_image = np.expand_dims(my_image, axis=0)

In [ ]:
my_image.shape

In [ ]:
model.predict(my_image)

In [ ]:
train_image_gen.class_indices

In [ ]:
test_image_gen.class_indices